# Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [2]:
import pandas as pd

In [21]:
ratings = pd.read_csv('Desktop/ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [22]:
results = ratings.groupby('userId').agg({'rating': 'count', 'timestamp': ['min', 'max']})
results = results.reset_index()
results.head()

userId rating   timestamp            
          count         min         max
0      1     20  1260759108  1260759205
1      2     76   835355395   835356246
2      3     51  1298861589  1298932787
3      4    204   949778714   949982274
4      5    100  1163373044  1163375145

In [23]:
results['lifetime'] = results['timestamp']['max'] - results['timestamp']['min']
results.head()

userId rating   timestamp             lifetime
          count         min         max         
0      1     20  1260759108  1260759205       97
1      2     76   835355395   835356246      851
2      3     51  1298861589  1298932787    71198
3      4    204   949778714   949982274   203560
4      5    100  1163373044  1163375145     2101

In [38]:
f"Ср. время жизни: {int(results[results['rating']['count'] >= 100]['lifetime'].mean()):,}"

'Ср. время жизни: 39,331,748'

# Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
- rzd - железнодорожные перевозки
- auto - автомобильные перевозки
- air - воздушные перевозки
- client_base - адреса клиентов

In [3]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [4]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [5]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [6]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [41]:
from functools import reduce

In [42]:
data_frames = [rzd, auto, air, client_base]

In [53]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['client_id'],
                                            how='outer'), data_frames).fillna(0)

In [54]:
df_merged.iloc[:,0:4]

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,0.0,0.0
1,112,2810.0,0.0,0.0
2,113,10283.0,57483.0,0.0
3,114,5774.0,83.0,0.0
4,115,981.0,912.0,81.0
5,116,0.0,4834.0,4.0
6,117,0.0,98.0,13.0
7,118,0.0,0.0,173.0


In [55]:
df_merged

,client_id,rzd_revenue,auto_revenue,air_revenue,address
0,111,1093.0,0.0,0.0,Комсомольская 4
1,112,2810.0,0.0,0.0,Энтузиастов 8а
2,113,10283.0,57483.0,0.0,Левобережная 1а
3,114,5774.0,83.0,0.0,Мира 14
4,115,981.0,912.0,81.0,ЗЖБИиДК 1
5,116,0.0,4834.0,4.0,Строителей 18
6,117,0.0,98.0,13.0,Панфиловская 33
7,118,0.0,0.0,173.0,Мастеркова 4


# Задание 3

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?

- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?

- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

In [94]:
logs = pd.read_csv('Desktop/visit_log.csv', sep = ";")
logs.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [121]:
# создаю несколько одинаковых таблиц, чтобы потом у 1 пользователя сгенерировалось несколько различных координат
users = pd.DataFrame(logs['user_id'].unique(), columns = ['user_id'])
users = pd.concat([users]*5, ignore_index=True)
users.tail()

,user_id
86480,98b19810d0
86481,32ebb20c13
86482,b85baa8c73
86483,e154b06121
86484,7d27a58fb8


In [122]:
from random import uniform

def longitude_assignment(row):
    x = uniform(-180,180)
    return x

def latitude_assignment(row):
    y = uniform(-90, 90)
    return y

In [123]:
users['longitude'] = users.apply(longitude_assignment, axis=1)
users['latitude'] = users.apply(latitude_assignment, axis=1)
users.tail()

,user_id,longitude,latitude
86480,98b19810d0,162.010364,9.518018
86481,32ebb20c13,171.542941,7.431326
86482,b85baa8c73,-82.319453,-4.825887
86483,e154b06121,83.046674,-68.817042
86484,7d27a58fb8,-20.734483,-16.646866


In [171]:
# 1-2 вопросы.
# для ответа на первый вопрос я бы посчитал наиболее часто встречаемое значение координат (mode) или же просто среднее знач.
# но ср. знач. менее предпочтительно.
grouped = users.groupby('user_id').agg({'longitude': 'mean', 'latitude': 'mean'})
grouped.head()

,longitude,latitude
user_id,,
000283210f,-30.811953,-8.282732
0007036410,69.798042,37.950484
0008dc82ee,95.721121,0.279321
000a519bab,-56.293098,-45.078118
000f452c0d,-22.414824,10.507228


In [ ]:
# 3 вопрос.
# можно сравнить координаты с географией нашего магазина/склада или посмотреть на частоту изменения координат